In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.color import rgb2gray
from skimage.transform import resize
from skimage.feature import graycomatrix, graycoprops, local_binary_pattern
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
from tqdm import tqdm
import pandas as pd
# Function to load and preprocess images
def load_dataset(base_path):
    images = []
    labels = []
    
    # Loop through each folder (type1, type2, ...)
    for class_idx, folder in enumerate(['type1', 'type2', 'type3', 'type4', 'type5']):
        folder_path = os.path.join(base_path, folder)
        print(f"Loading images from {folder_path}")
        
        # Loop through each image in the folder
        for filename in tqdm(os.listdir(folder_path)):
            if filename.endswith(('.jpg', '.jpeg', '.png')):
                img_path = os.path.join(folder_path, filename)
                
                # Read the image
                img = imread(img_path)
                
                # Convert to grayscale if the image is RGB
                if len(img.shape) > 2:
                    img = rgb2gray(img)
                
                # Resize to 256x256
                img = resize(img, (256, 256), anti_aliasing=True)
                
                # Normalize pixel values to [0, 1] if not already
                if img.max() > 1.0:
                    img = img / 255.0
                    
                # Add image and label to lists
                images.append(img)
                labels.append(class_idx)  # Use folder index as label
    
    return np.array(images), np.array(labels)

# Load the dataset
# Replace 'path_to_your_dataset' with the actual path to your dataset folder
dataset_path = '.'  # Current directory, update if needed
images, labels = load_dataset(dataset_path)

# Display some basic information about the dataset
print(f"Total number of images: {len(images)}")
print(f"Image shape: {images[0].shape}")
print(f"Number of classes: {len(np.unique(labels))}")
print(f"Class distribution: {np.bincount(labels)}")

# Visualize sample images from each class
plt.figure(figsize=(15, 10))
for class_idx in range(5):  # For each class
    # Get indices for this class
    class_indices = np.where(labels == class_idx)[0]
    
    # Select 5 random images from this class
    if len(class_indices) >= 5:
        sample_indices = np.random.choice(class_indices, 5, replace=False)
        
        # Plot each image
        for i, idx in enumerate(sample_indices):
            plt.subplot(5, 5, class_idx*5 + i + 1)
            plt.imshow(images[idx], cmap='gray')
            plt.title(f"Type {class_idx+1}")
            plt.axis('off')

plt.tight_layout()
plt.show()

# Split the dataset into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(
    images, labels, test_size=0.2, random_state=42, stratify=labels
)

print(f"Training set size: {X_train.shape[0]} images")
print(f"Testing set size: {X_test.shape[0]} images")
def extract_glcm_features(image):
    """
    Extract GLCM features from an image:
    - Contrast
    - Correlation
    - Energy
    - Homogeneity
    """
    # Convert floating-point image to uint8 for GLCM calculation
    # GLCM requires discrete gray levels
    image_uint8 = (image * 255).astype(np.uint8)
    
    # Calculate GLCM matrix
    # distances: [1] - one pixel distance
    # angles: [0, np.pi/4, np.pi/2, 3*np.pi/4] - 0°, 45°, 90°, 135°
    glcm = graycomatrix(image_uint8, 
                       distances=[1], 
                       angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], 
                       levels=256,
                       symmetric=True, 
                       normed=True)
    
    # Calculate GLCM properties
    contrast = graycoprops(glcm, 'contrast').mean()
    correlation = graycoprops(glcm, 'correlation').mean()
    energy = graycoprops(glcm, 'energy').mean()
    homogeneity = graycoprops(glcm, 'homogeneity').mean()
    
    # Return features as a list
    return [contrast, correlation, energy, homogeneity]

# Extract GLCM features for training set
print("Extracting GLCM features for training images...")
glcm_features_train = np.array([extract_glcm_features(img) for img in tqdm(X_train)])

# Extract GLCM features for testing set
print("Extracting GLCM features for testing images...")
glcm_features_test = np.array([extract_glcm_features(img) for img in tqdm(X_test)])

# Display feature shapes
print(f"GLCM training features shape: {glcm_features_train.shape}")
print(f"GLCM testing features shape: {glcm_features_test.shape}")

# Visualize the GLCM features
glcm_feature_names = ['Contrast', 'Correlation', 'Energy', 'Homogeneity']

# Create a DataFrame for easy visualization
glcm_df = pd.DataFrame(glcm_features_train, columns=glcm_feature_names)
glcm_df['Class'] = y_train

# Calculate the mean of each feature for each class
glcm_class_means = glcm_df.groupby('Class').mean()

# Plot the mean values
plt.figure(figsize=(12, 6))
glcm_class_means.plot(kind='bar', ax=plt.gca())
plt.title('Mean GLCM Features by Class')
plt.xlabel('Class')
plt.ylabel('Feature Value')
plt.legend(title='Features')
plt.tight_layout()
plt.show()

# Box plots for each feature across classes
plt.figure(figsize=(15, 10))
for i, feature in enumerate(glcm_feature_names):
    plt.subplot(2, 2, i+1)
    sns.boxplot(x='Class', y=feature, data=glcm_df)
    plt.title(f'Distribution of {feature} by Class')
plt.tight_layout()
plt.show()
def extract_lbp_features(image, num_points=8, radius=1, num_bins=10):
    """
    Extract LBP features from an image
    """
    # Compute LBP
    lbp = local_binary_pattern(image, num_points, radius, method='uniform')
    
    # Compute histogram
    n_bins = num_points + 2  # Number of bins for uniform LBP
    hist, _ = np.histogram(lbp.ravel(), bins=n_bins, range=(0, n_bins), density=True)
    
    return hist

# Extract LBP features for training set
print("Extracting LBP features for training images...")
lbp_features_train = np.array([extract_lbp_features(img) for img in tqdm(X_train)])

# Extract LBP features for testing set
print("Extracting LBP features for testing images...")
lbp_features_test = np.array([extract_lbp_features(img) for img in tqdm(X_test)])

# Display feature shapes
print(f"LBP training features shape: {lbp_features_train.shape}")
print(f"LBP testing features shape: {lbp_features_test.shape}")

# Visualize LBP on a sample image
plt.figure(figsize=(12, 6))

# Sample image index
sample_idx = 0

# Original image
plt.subplot(1, 2, 1)
plt.imshow(X_train[sample_idx], cmap='gray')
plt.title('Original Image')
plt.axis('off')

# LBP image
lbp_image = local_binary_pattern(X_train[sample_idx], 8, 1, method='uniform')
plt.subplot(1, 2, 2)
plt.imshow(lbp_image, cmap='gray')
plt.title('LBP Image')
plt.axis('off')

plt.tight_layout()
plt.show()

# Visualize LBP histograms for different classes
plt.figure(figsize=(15, 10))
for class_idx in range(5):
    # Get indices of images from this class
    indices = np.where(y_train == class_idx)[0]
    
    # Take the mean of LBP histograms for this class
    mean_hist = np.mean(lbp_features_train[indices], axis=0)
    
    # Plot histogram
    plt.subplot(2, 3, class_idx + 1)
    plt.bar(range(len(mean_hist)), mean_hist)
    plt.title(f'Mean LBP Histogram for Type {class_idx+1}')
    plt.xlabel('LBP Bin')
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()
# Combine GLCM and LBP features
print("Combining GLCM and LBP features...")
combined_features_train = np.hstack((glcm_features_train, lbp_features_train))
combined_features_test = np.hstack((glcm_features_test, lbp_features_test))

print(f"Combined training features shape: {combined_features_train.shape}")
print(f"Combined testing features shape: {combined_features_test.shape}")

# Standardize the features (important for SVM)
scaler = StandardScaler()
combined_features_train_scaled = scaler.fit_transform(combined_features_train)
combined_features_test_scaled = scaler.transform(combined_features_test)

# Initialize different classifiers
classifiers = {
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'SVM': SVC(kernel='rbf', probability=True, random_state=42)
}

# Train and evaluate classifiers
results = {}

for name, clf in classifiers.items():
    print(f"\nTraining {name} classifier...")
    
    # Train the classifier
    clf.fit(combined_features_train_scaled, y_train)
    
    # Make predictions
    y_pred = clf.predict(combined_features_test_scaled)
    
    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{name} Accuracy: {accuracy:.4f}")
    
    # Store results
    results[name] = {
        'classifier': clf,
        'predictions': y_pred,
        'accuracy': accuracy
    }

# Select the best classifier (based on accuracy)
best_clf_name = max(results, key=lambda k: results[k]['accuracy'])
best_clf = results[best_clf_name]['classifier']
best_predictions = results[best_clf_name]['predictions']

print(f"\nBest Classifier: {best_clf_name} with accuracy: {results[best_clf_name]['accuracy']:.4f}")

# Compare with individual feature sets
print("\nComparing with individual feature sets...")

# Train on GLCM features only
clf_glcm = SVC(kernel='rbf', probability=True, random_state=42)
glcm_features_train_scaled = scaler.fit_transform(glcm_features_train)
glcm_features_test_scaled = scaler.transform(glcm_features_test)
clf_glcm.fit(glcm_features_train_scaled, y_train)
glcm_accuracy = accuracy_score(y_test, clf_glcm.predict(glcm_features_test_scaled))
print(f"GLCM Features Only Accuracy: {glcm_accuracy:.4f}")

# Train on LBP features only
clf_lbp = SVC(kernel='rbf', probability=True, random_state=42)
lbp_features_train_scaled = scaler.fit_transform(lbp_features_train)
lbp_features_test_scaled = scaler.transform(lbp_features_test)
clf_lbp.fit(lbp_features_train_scaled, y_train)
lbp_accuracy = accuracy_score(y_test, clf_lbp.predict(lbp_features_test_scaled))
print(f"LBP Features Only Accuracy: {lbp_accuracy:.4f}")

# Compare accuracies
accuracies = {
    'GLCM Only': glcm_accuracy,
    'LBP Only': lbp_accuracy,
    'Combined': results[best_clf_name]['accuracy']
}

plt.figure(figsize=(10, 6))
plt.bar(accuracies.keys(), accuracies.values())
plt.ylim(0, 1.0)
plt.xlabel('Feature Set')
plt.ylabel('Accuracy')
plt.title('Classification Accuracy Comparison')
for i, v in enumerate(accuracies.values()):
    plt.text(i, v + 0.01, f"{v:.4f}", ha='center')
plt.show()
# Generate detailed classification report
print("\nClassification Report:")
report = classification_report(y_test, best_predictions, target_names=[f'Type {i+1}' for i in range(5)])
print(report)

# Generate confusion matrix
cm = confusion_matrix(y_test, best_predictions)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=[f'Type {i+1}' for i in range(5)],
            yticklabels=[f'Type {i+1}' for i in range(5)])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

# Calculate per-class metrics
class_names = [f'Type {i+1}' for i in range(5)]
precision = np.diag(cm) / np.sum(cm, axis=0)
recall = np.diag(cm) / np.sum(cm, axis=1)
f1_score = 2 * (precision * recall) / (precision + recall)

# Plot per-class metrics
metrics_df = pd.DataFrame({
    'Precision': precision,
    'Recall': recall,
    'F1-Score': f1_score
}, index=class_names)

plt.figure(figsize=(12, 6))
metrics_df.plot(kind='bar', ax=plt.gca())
plt.title('Per-Class Metrics')
plt.xlabel('Class')
plt.ylabel('Score')
plt.ylim(0, 1.0)
plt.legend(title='Metrics')
plt.tight_layout()
plt.show()

# Feature importance analysis (for Random Forest)
# FIXED: Corrected access to the Random Forest classifier
if 'Random Forest' in results:  # Changed from 'classifiers' to 'results'
    rf_clf = results['Random Forest']['classifier']  # Corrected dictionary access
    
    # Combine feature names
    feature_names = glcm_feature_names + [f'LBP_{i}' for i in range(lbp_features_train.shape[1])]
    
    # Get feature importances
    importances = rf_clf.feature_importances_
    
    # Sort features by importance
    indices = np.argsort(importances)[::-1]
    
    # Plot top 10 features
    plt.figure(figsize=(12, 6))
    plt.title('Feature Importances')
    plt.bar(range(min(10, len(indices))), importances[indices[:10]], align='center')
    plt.xticks(range(min(10, len(indices))), [feature_names[i] for i in indices[:10]], rotation=45)
    plt.tight_layout()
    plt.show()

def display_predictions(X_test, y_test, y_pred, num_samples=10):
    """
    Display random test images along with their true and predicted classes
    """
    # Choose random indices
    indices = np.random.choice(range(len(y_test)), min(num_samples, len(y_test)), replace=False)
    
    # Create a figure
    plt.figure(figsize=(15, 2 * num_samples))
    
    # For each selected index
    for i, idx in enumerate(indices):
        # Get the image and labels
        image = X_test[idx]
        true_label = y_test[idx]
        pred_label = y_pred[idx]
        
        # Display the image
        plt.subplot(num_samples, 2, 2*i + 1)
        plt.imshow(image, cmap='gray')
        plt.title(f"True: Type {true_label+1}")
        plt.axis('off')
        
        # Show the same image with predicted label
        plt.subplot(num_samples, 2, 2*i + 2)
        plt.imshow(image, cmap='gray')
        
        # Highlight correct/incorrect predictions
        if true_label == pred_label:
            color = 'green'
        else:
            color = 'red'
            
        plt.title(f"Predicted: Type {pred_label+1}", color=color)
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()

# Display predictions for some random test images
print("Visualizing predictions on random test images:")
display_predictions(X_test, y_test, best_predictions, num_samples=8)

# Calculate and display overall accuracy
accuracy = accuracy_score(y_test, best_predictions)
print(f"Overall accuracy: {accuracy:.4f}")

# Display a few examples of misclassifications
misclassified_indices = np.where(y_test != best_predictions)[0]

if len(misclassified_indices) > 0:
    print("\nExamples of misclassified images:")
    
    # Choose random misclassified samples (up to 5)
    num_examples = min(5, len(misclassified_indices))
    sample_indices = np.random.choice(misclassified_indices, num_examples, replace=False)
    
    plt.figure(figsize=(15, 3 * num_examples))
    
    for i, idx in enumerate(sample_indices):
        # Display the misclassified image
        plt.subplot(num_examples, 1, i + 1)
        plt.imshow(X_test[idx], cmap='gray')
        plt.title(f"True: Type {y_test[idx]+1}, Predicted: Type {best_predictions[idx]+1}")
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()
else:
    print("No misclassifications found!")
## Conclusion

#### This project demonstrated the effectiveness of combining GLCM and LBP features for texture classification. The system achieved high accuracy in classifying different fabric textures, showing its potential for real-world applications in the textile industry. Future work could explore the integration of deep learning approaches to further improve classification performance.